# Segmenting and Clustering Neighbourhoods in Toronto

## Import required libraries

In [27]:
# general imports
import pandas as pd
import numpy as np

# required for pulling down web data
import urllib

# required for parsing an HTML document
from bs4 import BeautifulSoup

## Pull down a copy of the data

In [28]:
# obtain a copy of the web page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

## Parse the downloaded data

In [29]:
# parse the downloaded HTML page
soup = BeautifulSoup(page, 'html.parser')

#get a reference to the correct table
table=soup.find('table', class_='wikitable sortable')

## Extract the column names

In [30]:
#obtain the column names, stripping off whitespace
columnNames = []
for name in table.findAll( 'th' ):
    columnNames.append( name.find(text=True ).strip() )

#lets see what we've got    
print( columnNames )

['Postcode', 'Borough', 'Neighbourhood']


## Extract the data into a DataFrame

In [31]:
#location to store the data during the extraction loop
cellData = []
rowData = []

#pull out each row of data
for row in table.findAll( 'tr' ):

    #pull out each cell from the row
    for cell in row.findAll( 'td' ):
        cellData.append( cell.find(text=True).strip() )

    #only append if we hold complete and correct data (exclude if borough is 'Not assigned')
    if( len( cellData ) == len( columnNames ) ):
        if( cellData[1] != "Not assigned" ):
            rowData.append( cellData )
    cellData = []


#once we have all the cells, create a data frame with this data
df = pd.DataFrame( np.array( rowData ), columns=columnNames )

## Combine Neighbourhoods

In [32]:
#group the data by post code, combining Neighbourhood 
grouped = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join ).reset_index()

# Ensure we print out the same order as the given screenshot - not required, just for fun
grouped.loc[ (grouped.Postcode == 'M5G') | (grouped.Postcode == 'M2H') | (grouped.Postcode == 'M4B')| (grouped.Postcode == 'M1J')| (grouped.Postcode == 'M4G')| (grouped.Postcode == 'M4M')| (grouped.Postcode == 'M1R')| (grouped.Postcode == 'M9V')| (grouped.Postcode == 'M9L')| (grouped.Postcode == 'M5V')| (grouped.Postcode == 'M1B')| (grouped.Postcode == 'M5A')]

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
5,M1J,Scarborough,Scarborough Village
11,M1R,Scarborough,"Maryvale,Wexford"
17,M2H,North York,Hillcrest Village
35,M4B,East York,"Woodbine Gardens,Parkview Hill"
38,M4G,East York,Leaside
43,M4M,East Toronto,Studio District
53,M5A,Downtown Toronto,"Harbourfront,Regent Park"
57,M5G,Downtown Toronto,Central Bay Street
68,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf..."


## Replace 'Not Assigned' in Neighbourhoods

In [33]:
grouped.shape

(103, 3)

In [34]:
#validate we have a 'Not assigned' neighbourhood
print( grouped.loc[grouped.Neighbourhood == 'Not assigned'] )
print()

#when the neighbourhood is 'Not assigned', replace with the borough
grouped.Neighbourhood = pd.np.where(grouped.Neighbourhood == 'Not assigned', grouped.Borough, grouped.Neighbourhood)

#validate we do NOT have a 'Not assigned' neighbourhood
print( grouped.loc[grouped.Neighbourhood == 'Not assigned'])
print()

   Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Not assigned

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []



In [35]:
geospatial = {
'Postcode' : [ 'M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M', 'M1N', 'M1P', 'M1R', 'M1S', 'M1T', 'M1V', 'M1W', 'M1X', 'M2H', 'M2J', 'M2K', 'M2L', 'M2M', 'M2N', 'M2P', 'M2R', 'M3A', 'M3B', 'M3C', 'M3H', 'M3J', 'M3K', 'M3L', 'M3M', 'M3N', 'M4A', 'M4B', 'M4C', 'M4E', 'M4G', 'M4H', 'M4J', 'M4K', 'M4L', 'M4M', 'M4N', 'M4P', 'M4R', 'M4S', 'M4T', 'M4V', 'M4W', 'M4X', 'M4Y', 'M5A', 'M5B', 'M5C', 'M5E', 'M5G', 'M5H', 'M5J', 'M5K', 'M5L', 'M5M', 'M5N', 'M5P', 'M5R', 'M5S', 'M5T', 'M5V', 'M5W', 'M5X', 'M6A', 'M6B', 'M6C', 'M6E', 'M6G', 'M6H', 'M6J', 'M6K', 'M6L', 'M6M', 'M6N', 'M6P', 'M6R', 'M6S', 'M7A', 'M7R', 'M7Y', 'M8V', 'M8W', 'M8X', 'M8Y', 'M8Z', 'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N', 'M9P', 'M9R', 'M9V', 'M9W' ],
'Latitude' : [ '43.8066863', '43.7845351', '43.7635726', '43.7709921', '43.773136', '43.7447342', '43.7279292', '43.7111117', '43.716316', '43.692657', '43.7574096', '43.7500715', '43.7942003', '43.7816375', '43.8152522', '43.7995252', '43.8361247', '43.8037622', '43.7785175', '43.7869473', '43.7574902', '43.789053', '43.7701199', '43.7527583', '43.7827364', '43.7532586', '43.7459058', '43.7258997', '43.7543283', '43.7679803', '43.7374732', '43.7390146', '43.7284964', '43.7616313', '43.7258823', '43.7063972', '43.6953439', '43.6763574', '43.7090604', '43.7053689', '43.685347', '43.6795571', '43.6689985', '43.6595255', '43.7280205', '43.7127511', '43.7153834', '43.7043244', '43.6895743', '43.6864123', '43.6795626', '43.667967', '43.6658599', '43.6542599', '43.6571618', '43.6514939', '43.6447708', '43.6579524', '43.6505712', '43.6408157', '43.6471768', '43.6481985', '43.7332825', '43.7116948', '43.6969476', '43.6727097', '43.6626956', '43.6532057', '43.6289467', '43.6464352', '43.6484292', '43.718518', '43.709577', '43.6937813', '43.6890256', '43.669542', '43.6690051', '43.6479267', '43.6368472', '43.7137562', '43.6911158', '43.6731853', '43.6616083', '43.6489597', '43.6515706', '43.6623015', '43.6369656', '43.6627439', '43.6056466', '43.6024137', '43.6536536', '43.6362579', '43.6288408', '43.6678556', '43.6509432', '43.6435152', '43.7563033', '43.7247659', '43.706876', '43.696319', '43.6889054', '43.7394164', '43.7067483' ],
'Longitude' : [ '-79.1943534', '-79.1604971', '-79.1887115', '-79.2169174', '-79.2394761', '-79.2394761', '-79.2620294', '-79.2845772', '-79.2394761', '-79.2648481', '-79.273304', '-79.2958491', '-79.2620294', '-79.3043021', '-79.2845772', '-79.3183887', '-79.2056361', '-79.3634517', '-79.3465557', '-79.385975', '-79.3747141', '-79.4084928', '-79.4084928', '-79.4000493', '-79.4422593', '-79.3296565', '-79.352188', '-79.340923', '-79.4422593', '-79.4872619', '-79.4647633', '-79.5069436', '-79.4956974', '-79.5209994', '-79.3155716', '-79.309937', '-79.3183887', '-79.2930312', '-79.3634517', '-79.3493719', '-79.3381065', '-79.352188', '-79.3155716', '-79.340923', '-79.3887901', '-79.3901975', '-79.4056784', '-79.3887901', '-79.3831599', '-79.4000493', '-79.3775294', '-79.3676753', '-79.3831599', '-79.3606359', '-79.3789371', '-79.3754179', '-79.3733064', '-79.3873826', '-79.3845675', '-79.3817523', '-79.3815764', '-79.3798169', '-79.4197497', '-79.4169356', '-79.4113072', '-79.4056784', '-79.4000493', '-79.4000493', '-79.3944199', '-79.374846', '-79.3822802', '-79.4647633', '-79.4450726', '-79.4281914', '-79.453512', '-79.4225637', '-79.4422593', '-79.4197497', '-79.4281914', '-79.4900738', '-79.4760133', '-79.4872619', '-79.4647633', '-79.456325', '-79.4844499', '-79.3894938', '-79.615819', '-79.321558', '-79.5013207', '-79.5434841', '-79.5069436', '-79.4985091', '-79.5209994', '-79.5322424', '-79.5547244', '-79.5772008', '-79.5659633', '-79.5322424', '-79.5181884', '-79.5322424', '-79.5547244', '-79.5884369', '-79.5940544' ]
}

geospatial_df = pd.DataFrame( geospatial, columns=['Postcode', 'Latitude', 'Longitude'] )

#join this geo data with the existing dataframe
joined_df = pd.merge( grouped, geospatial_df, on='Postcode', how='outer')

# Ensure we print out the same order as the given screenshot - not required, just for fun
joined_df.loc[ (joined_df.Postcode == 'M5G') | (joined_df.Postcode == 'M2H') | (joined_df.Postcode == 'M4B')| (joined_df.Postcode == 'M1J')| (joined_df.Postcode == 'M4G')| (joined_df.Postcode == 'M4M')| (joined_df.Postcode == 'M1R')| (joined_df.Postcode == 'M9V')| (joined_df.Postcode == 'M9L')| (joined_df.Postcode == 'M5V')| (joined_df.Postcode == 'M1B')| (joined_df.Postcode == 'M5A')]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8066863,-79.1943534
5,M1J,Scarborough,Scarborough Village,43.7447342,-79.2394761
11,M1R,Scarborough,"Maryvale,Wexford",43.7500715,-79.2958491
17,M2H,North York,Hillcrest Village,43.8037622,-79.3634517
35,M4B,East York,"Woodbine Gardens,Parkview Hill",43.7063972,-79.309937
38,M4G,East York,Leaside,43.7090604,-79.3634517
43,M4M,East Toronto,Studio District,43.6595255,-79.340923
53,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6542599,-79.3606359
57,M5G,Downtown Toronto,Central Bay Street,43.6579524,-79.3873826
68,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.6289467,-79.3944199


# START OF NOTEBOOK 2

---

## Combine geospatial data with the existing DataFrame